# Output Parsers

    - Output parsers emerged as an early solution to the challenge of obtaining structured output from LLMs.

    - Today, most LLMs support structured output natively. In such cases, using output parsers may be unnecessary, and you should leverage the model's built-in capabilities for structured output. Refer to the documentation of your chosen model for guidance on how to achieve structured output directly.

    - Output parsers remain valuable when working with models that do not support structured output natively, or when you require additional processing or validation of the model's output beyond its inherent capabilities.

    - Json Output Parser
    - StrOutputParser
    - StructuredOutputParser
    - PydanticOutputParser

In [1]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="qwen2.5-coder:7b",
    model_provider="ollama",
    temperature = 0.0
)

## StrOutputParser

    - The StrOutputParser is the simplest output parser in Langchain. It is used to parse the output of a LLM and return it as a plain string.

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = PromptTemplate(
    template="you are a helpfull assistant, who answer in {output_limit} words, \n {user_input}",
    input_variables=['output_limit','user_input']
)

query = 'explain about earth'
str_parser = StrOutputParser()

chain = template | model | str_parser

response = chain.invoke({
    'output_limit' : 20,
    'user_input' : query
})

print()
print(response)
print(type(response))


Earth is the third planet from the Sun, with water covering 71% of its surface. It's known for its diverse ecosystems and is home to millions of species.
<class 'langchain_core.messages.base.TextAccessor'>


## Json Output Parser

    - Cannot Enforce Schema

In [9]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate


parser = JsonOutputParser()

template = PromptTemplate(template="give me the name, age and city of fictional indian person \n {format_instruction}",
                          input_variables=[],
                          partial_variables={"format_instruction":parser.get_format_instructions()})

json_parser = JsonOutputParser()

chain = template | model | json_parser
response = chain.invoke({})

print(response)



{'name': 'Samantha Patel', 'age': 30, 'city': 'Mumbai'}


In [10]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate


parser = JsonOutputParser()

template = PromptTemplate(template="give me 5 facts about black hole \n {format_instruction}",
                          input_variables=[],
                          partial_variables={"format_instruction":parser.get_format_instructions()})

json_parser = JsonOutputParser()

chain = template | model | json_parser
response = chain.invoke({})

print(response)



{'black_hole_facts': [{'fact': 'Black holes are regions in space where the gravitational pull is so strong that nothing, not even light, can escape from them.'}, {'fact': 'The event horizon of a black hole is the boundary beyond which nothing can escape its gravity. It marks the point of no return for anything falling into the black hole.'}, {'fact': 'Black holes form when massive stars collapse in on themselves after they have exhausted their nuclear fuel. The core of these stars collapses under its own gravity, forming a singularity at the center.'}, {'fact': 'The first image of a black hole was captured by the Event Horizon Telescope (EHT) in 2019, showing the shadow cast by the supermassive black hole at the heart of the galaxy M87.'}, {'fact': 'Black holes can have varying masses, from small stellar black holes with masses similar to that of our Sun up to supermassive black holes found in the centers of galaxies, which can be millions or even billions of times more massive than th

## StructuredOutputParser

    - Can enforce schema
    - No data validation
    - Helps to extract structured JSON data from LLM responses based on predefined field schemas.
    - It works by defining a list of fields(Response schema) that the model should return, ensuring the output follows a structured format.

## PydanticOutputParser

    - Strict Schema Enforcement
    - Type Safety
    - Easy Validation
    - Seamless Integration

In [13]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class Personschema(BaseModel):
    name : str = Field(description='name of the person')
    age : int = Field(gt=18, description='age of the person')
    city : str = Field(description='name of thee city of the person belongs to')

parser = PydanticOutputParser(pydantic_object=Personschema)

template = PromptTemplate(
    template='Generate name, age, city of fictional indian person \n {format_instruction}',
    input_variables=[],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser
result = chain.invoke({})
print(result)


name='Samantha Patel' age=28 city='Mumbai'
